In [17]:
import requests
import json
import csv
import multiprocessing as mp
from itertools import repeat
import os
import time
import random
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import folium
import sqlalchemy
import tables
import pickle
from pathlib import Path
from collections import OrderedDict
import numpy as np
import pandas as pd
import shapely
import geopandas as gpd
from datetime import datetime, timedelta, time, date
from geopy.distance import vincenty, great_circle
from sqlalchemy.orm import sessionmaker #Run pip install sqlalchemy
from sklearn.cluster import DBSCAN
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelBinarizer, LabelEncoder



%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True)

# os.chdir('src')

import settings as s
import utility_io
import utility_database
import prediction_gps_grid_mode_topic
np.random.seed(1)
random.seed(1)

pd.set_option('display.width',200)



In [ ]:
engine, conn, metadata = utility_database.establish_db_connection_postgresql_geotweet_ssh()

In [6]:
# Check model file and topic feature file

EXPERIMENT_PARAMETERS = s.EXPERIMENT_PARAMETERS
LSI_MODEL_FILE = s.LSI_MODEL_FILE
LDA_MODEL_FILE = s.LDA_MODEL_FILE
DOC2VEC_MODEL_FILE = s.DOC2VEC_MODEL_FILE
STOPLIST_FILE = s.STOPLIST_FILE
DICT_FILE = s.DICT_FILE
MM_CORPUS_FILE = s.MM_CORPUS_FILE
TFIDF_FILE = s.TFIDF_FILE
LSI_TOPIC_FILE = s.LSI_TOPIC_FILE
LDA_TOPIC_FILE = s.LDA_TOPIC_FILE
DOC2VEC_TOPIC_FILE = s.DOC2VEC_TOPIC_FILE

dictionary = gensim.corpora.Dictionary.load(DICT_FILE)
lsi = gensim.models.LsiModel.load(LSI_MODEL_FILE)
lda = gensim.models.LdaModel.load(LDA_MODEL_FILE)
doc2vec = Doc2Vec.load(DOC2VEC_MODEL_FILE)
tfidf = gensim.models.TfidfModel.load(TFIDF_FILE)

FIGURE_DIR = s.FIGURE_DIR

X_GRID_FILE = s.X_GRID_FILE
Y_GRID_FILE = s.Y_GRID_FILE
X_MODE_FILE = s.X_MODE_FILE
Y_MODE_FILE = s.Y_MODE_FILE
X_TOPIC_FILE = s.X_TOPIC_FILE
Y_TOPIC_FILE = s.Y_TOPIC_FILE

LE_GRID_CLASSES_FILE = s.LE_GRID_CLASSES_FILE
LB_MODE_CLASSES_FILE = s.LB_MODE_CLASSES_FILE
X_FILE = s.X_FILE
Y_FILE = s.Y_FILE
Y_FILE_PREDICTED_LSTM = s.Y_FILE_PREDICTED_LSTM
Y_FILE_PREDICTED_VELOCITY = s.Y_FILE_PREDICTED_VELOCITY
MODEL_FILE_LSTM_GRID = s.MODEL_FILE_LSTM_GRID
MODEL_WEIGHT_FILE_LSTM_GRID = s.MODEL_WEIGHT_FILE_LSTM_GRID
GEOJSON_FILE_OBSERVATION_GRID = s.GEOJSON_FILE_OBSERVATION_GRID
GEOJSON_FILE_TRUE_GRID = s.GEOJSON_FILE_TRUE_GRID
GEOJSON_FILE_PREDICTED_LSTM_GRID = s.GEOJSON_FILE_PREDICTED_LSTM_GRID

le_grid = LabelEncoder()
le_grid.classes_ = np.load(LE_GRID_CLASSES_FILE)
lb_mode = LabelBinarizer()
lb_mode.classes_ = np.load(LB_MODE_CLASSES_FILE)


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
